In [1]:
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
# Credentials
DB_NAME = 'documentcoder'
DB_USER = 'postgres'
DB_PASS = 'save'

In [3]:
# CONNECTION IS ESTABLISHED IF NOTHING RETURNS 
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [4]:
# Create Cursor to execute statements
cur = conn.cursor()

In [13]:
# how many policies are in the dataset 
cur.execute('SELECT COUNT(*) FROM (SELECT DISTINCT id FROM api_policy) AS unique_policy_id_cnt;')
policy_cnt = cur.fetchall()[0][0]
print(policy_cnt)

628


In [14]:
# how many coding instances
cur.execute('SELECT COUNT(*) FROM (SELECT DISTINCT id FROM api_codinginstance) AS unique_codinginstance_cnt;')
coding_instance_cnt = cur.fetchall()[0][0]
print(coding_instance_cnt)

32


In [17]:
# how many policies with two coding instances
cur.execute("""SELECT COUNT(*)
            FROM (SELECT policy_instance_id, count(DISTINCT coder_email) AS coder_cnt
            FROM api_codinginstance GROUP BY policy_instance_id) AS sub
            WHERE sub.coder_cnt > 1;""")
cnt_policies_with_two_inst = cur.fetchall()[0][0]
print(cnt_policies_with_two_inst)

9


In [21]:
# how many policies with two coding instances AND corrections by FMW
cur.execute("""SELECT api_codinginstance.policy_instance_id, 
            COUNT(DISTINCT coder_email) as coder_cnt
            FROM api_codinginstance
            INNER JOIN
            (SELECT policy_instance_id
            FROM api_codinginstance
            WHERE coder_email = 'florencia.m.wurgler@gmail.com') AS fmw_ids
            ON fmw_ids.policy_instance_id = api_codinginstance.policy_instance_id
            WHERE coder_email != 'davidbstein@gmail.com'
            GROUP BY api_codinginstance.policy_instance_id
            HAVING COUNT(DISTINCT coder_email) > 2;""")
cnt_policies_with_two_inst_and_fmw = len(cur.fetchall())
print(cnt_policies_with_two_inst_and_fmw)

4
